<a href="https://colab.research.google.com/github/cbwang2016/normalizing_flow_tests/blob/main/coulomb_gas_realnvp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
import jax
from jax.config import config   
config.update("jax_enable_x64", True)
import jax.numpy as jnp
from jax import lax
from jax.experimental import ode 
from jax.experimental import stax
from jax.experimental import optimizers
from jax.experimental.stax import Dense, Flatten, Softplus, Relu
from jax.nn.initializers import normal
import math 
import numpy as np
import matplotlib.pyplot as plt 
from IPython import display

In [120]:
!nvidia-smi
jax.devices()

Thu Mar  3 08:36:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    76W / 149W |  10368MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

[GpuDevice(id=0, process_index=0)]

We'd like to study thermodynamic property of the classcial Coulomb gas, whose Hamiltonian reads

$$H= \sum_{i<j} \frac{1}{|\boldsymbol{x}_i - \boldsymbol{x}_j|} + \sum_i  \frac{\boldsymbol{x}_i^2 }{2}. $$
The second term is a harmonic trapping potential. It makes our story easier (no need to consider periodic bondary condition or Ewald sum for long range interaction.)

The way to go is to minimize the variationial free energy with respect to a variational probability density $p(\boldsymbol{x})$

$$\mathcal{L} = \mathbb{E}_{\boldsymbol{x} \sim p(\boldsymbol{x})} \left [ \ln p(\boldsymbol{x}) + \beta H(\boldsymbol{x}) \right] \ge -\ln Z, $$ 
where $Z = \int d \boldsymbol{x} e^{-\beta H}$ and $\beta$ is the inverse temperature. The equality holds when $p(\boldsymbol{x}) = e^{-\beta H}/Z$, i.e., we achieve the exact solution. 



First thing first, here is the energy function

In [121]:
def energy(x, n, dim):
    i, j = np.triu_indices(n, k=1)
    r_ee = jnp.linalg.norm((jnp.reshape(x, (n, 1, dim)) - jnp.reshape(x, (1, n, dim)))[i,j], axis=-1) 
    v_ee = jnp.sum(1/r_ee)
    return jnp.sum(x**2/2) + v_ee 

The probablistic model we are going to use is the continuous normaliznig flow https://arxiv.org/abs/1806.07366. This is not going to be the most effecient approach. But it is certainly fun! 

In a nutshell, we integrate these two equations to transform samples and their log-likelihoods:

\begin{eqnarray}
\frac{d \boldsymbol{x} }{d t } & =   \boldsymbol{v}, \tag{1}  \\
\frac{d \ln p (\boldsymbol{x}) }{d t} &  = -\nabla \cdot \boldsymbol{v} \tag{2}. 
\end{eqnarray}

Here $\boldsymbol{v}$ is a velocity field we parametrize using a neural network.


In [122]:
def nvp_forward(net_params, shift_and_log_scale_fn, x, flip=False):
    d = x.shape[-1]//2
    x1, x2 = x[:, :d], x[:, d:]
    if flip:
        x2, x1 = x1, x2
    shift, log_scale = shift_and_log_scale_fn(net_params, x1)
    y2 = x2*jnp.exp(log_scale) + shift
    if flip:
        x1, y2 = y2, x1
    y = jnp.concatenate([x1, y2], axis=-1)
    return y

def nvp_inverse(net_params, shift_and_log_scale_fn, y, flip=False):
    d = y.shape[-1]//2
    y1, y2 = y[:, :d], y[:, d:]
    if flip:
        y1, y2 = y2, y1
    shift, log_scale = shift_and_log_scale_fn(net_params, y1)
    x2 = (y2-shift)*jnp.exp(-log_scale)
    if flip:
        y1, x2 = x2, y1
    x = jnp.concatenate([y1, x2], axis=-1)
    return x, log_scale


def flow(ps, configs, x):
    '''
    flow from z to x
    '''
    for p, config in zip(ps, configs):
        shift_log_scale_fn, flip = config
        x = nvp_forward(p, shift_log_scale_fn, x, flip=flip)
    return x

def logp(ps, configs, y):
    '''
    likelihood of given samples
    '''
    
    def base_logp(x):
        return -0.5 * jnp.sum( x**2 + jnp.log(2 * math.pi), axis=-1)

    rtn = 0
    for p, config in list(zip(ps, configs))[::-1]:
        shift_log_scale_fn, flip = config
        y, log_scale = nvp_inverse(p, shift_log_scale_fn, y, flip=flip)
        rtn -= jnp.sum(log_scale, axis=-1)
    rtn += base_logp(y)
    
    return rtn



Gradient of the objective function is

$$\nabla \mathcal{L} = \mathbb{E}_{\boldsymbol{x} \sim p(\boldsymbol{x})} \left[ f (\boldsymbol{x})  \nabla \ln p(\boldsymbol{x}) \right],$$ 
where $f (\boldsymbol{x}) = \ln p(\boldsymbol{x}) + \beta H(\boldsymbol{x})$. This is known as the REINFORCE algorithm, or score function gradient estimator. See https://arxiv.org/abs/1906.10652 for more details. 


In [123]:
def make_reinforce_loss(cs, n, dim, beta):

    batch_energy = jax.vmap(energy, (0, None, None), 0) 
    # batch_logp = jax.vmap(logp, (None, None, 0), 0) 

    def loss(params, x):
    
        entropy = logp(params, cs, x)

        f = entropy + beta*batch_energy(x, n, dim) 
        f = jax.lax.stop_gradient(f)

        f_mean = jnp.mean(f)

        f_std = jnp.std(f)/jnp.sqrt(f.shape[0])

        return jnp.mean((f - f_mean) * entropy), (f_mean, f_std)

    return loss


Now we are ready to initialize a network for the velocity field $\boldsymbol{v}$

In [124]:
batchsize = 8192
n = 6
dim = 2 
beta = 5.0
key = jax.random.PRNGKey(42)
key, subkey = jax.random.split(key, 2)

def init_nvp(D):
    net_init, net_apply = stax.serial(
        Dense(2048), Relu, Dense(2048), Relu, Dense(D))
    in_shape = (-1, D//2)
    out_shape, net_params = net_init(subkey, in_shape)
    # print(out_shape)
    def shift_and_log_scale_fn(net_params, x1):
        s = net_apply(net_params, x1)
        return np.split(s, 2, axis=1)
    return net_params, shift_and_log_scale_fn

def init_nvp_chain(D=6, n=8):
    flip = False
    ps, configs = [], []
    for i in range(n):
        p, f = init_nvp(D)
        ps.append(p), configs.append((f, flip))
        flip = not flip
    return ps, configs


ps0, cs = init_nvp_chain(n * dim)

loss = make_reinforce_loss(cs, n, dim, beta)
value_and_grad = jax.value_and_grad(loss, argnums=0, has_aux=True)

opt_init, opt_update, get_params = optimizers.adam(step_size=1e-4)
opt_state = opt_init(ps0)

# in_shape = (-1, n*dim)
# key, subkey = jax.random.split(key, 2)
# out_shape, params = net_init(subkey, in_shape)

Here is the training loop. During training we monitor the density and loss histroy. 

In [125]:
# ps, cs = init_nvp_chain(n * dim)
# x = jax.random.normal(subkey, (batchsize, n*dim)) 
# y = flow(ps, cs, x)
# def tmp(ps, configs, y):
#     for p, config in list(zip(ps, configs))[::-1]:
#         shift_log_scale_fn, flip = config
#         y, log_scale = nvp_inverse(p, shift_log_scale_fn, y, flip=flip)
    
#     return y
# print(np.max(np.abs(np.array(x - tmp(ps, cs, y)))))
# # print(ps)

In [ ]:
def step(i, subkey, opt_state):
    params = get_params(opt_state)
    
    x = jax.random.normal(subkey, (batchsize, n*dim)) 
    # x = jax.vmap(flow, (None, None, 0), 0)(params, network, x)
    x = flow(params, cs, x)

    x = jax.lax.stop_gradient(x)

    value, grad = value_and_grad(params, x)

    opt_state = opt_update(i, grad, opt_state)
    return value, opt_state, x

loss_histroy = []
for i in range(500):
    # key, subkey = jax.random.split(key)
    value, opt_state, x = step(i, subkey, opt_state)
    _, (f_mean, f_err) = value

    loss_histroy.append([f_mean, f_err])
    print([u[0] for u in loss_histroy])
   
    display.clear_output(wait=True)

    fig = plt.figure(figsize=(12, 6))

    #v = network(get_params(opt_state), x[0]) # velocity on the first sample
    #v = jnp.reshape(v, (n, dim))
    x = jnp.reshape(x, (batchsize*n, dim)) 

    display.clear_output(wait=True)

    fig = plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    #plt.plot(x[:, 0], x[:, 1],'o', alpha=0.5)
    #plt.quiver(x[:, 0], x[:, 1], v[:, 0], v[:, 1])

    #density plot
    H, xedges, yedges = np.histogram2d(x[:, 0], x[:, 1], 
                                       bins=100, 
                                       range=((-4, 4), (-4, 4)),
                        density=True)
    plt.imshow(H, interpolation="nearest", 
               extent=(xedges[0], xedges[-1], yedges[0], yedges[-1]),
               cmap="inferno")

    plt.xlim([-4, 4])
    plt.ylim([-4, 4])

    plt.subplot(1,2,2)
    y = np.reshape(np.array(loss_histroy), (-1, 2))
    plt.errorbar(np.arange(i+1), y[:, 0], yerr=y[:, 1], marker='o', capsize=8)
    plt.xlabel('epochs')
    plt.ylabel('variational free energy')
    plt.pause(0.01)


<Figure size 864x432 with 0 Axes>

Do you see structure emerges from training? 
Yes! It is called [Wigner molecule](https://en.wikipedia.org/wiki/Wigner_crystal). Just think about a few marbles in a bow. You got it. 

A few things to improve from this mininist implementation: 
* It seems the training breaks rotational invariance. But it shouldn't. How to incorporate that into the model design ? 

* How about permutation equivariance $\mathcal{P} \boldsymbol{v}(\boldsymbol{x}) = v(\mathcal{P} \boldsymbol{x})$ ?

* Do you feel it is a bit ackward to integrate the equation twice (one for sampling, one for the likelihood). It is actually easier to implement "reparametrization" gradient estimator. Code it and see why we were not using it here. 

* This code is painfully slow for such a tiny problem. This is mainly because we have repeatedly computed divergence $\nabla \cdot \boldsymbol{v}$ of the neural network. See https://arxiv.org/abs/1810.01367 and https://arxiv.org/abs/1912.03579 for ideas of speeding up.






# sandbox

In [ ]:
def divergence_scan_jvp(f):
    def _div_f(params, x):
        n = x.shape[0]
        eye = jnp.eye(n)

        def _body_fun(val, i):
            primal, tangent = jax.jvp(lambda x: f(params, x), (x,), (eye[i],))
            return val + tangent[i], None
                                             
        return lax.scan(_body_fun, 0.0, jnp.arange(0, n))[0]
    return _div_f

def divergence_scan_vjp(f):
    def _div_f(params, x):
        n = x.shape[0]
        eye = jnp.eye(n)

        def _body_fun(val, i):
            primal, vjp = jax.vjp(lambda x: f(params, x), x)
            return val + vjp(eye[i])[0][i], None
                                             
        return lax.scan(_body_fun, 0.0, jnp.arange(0, n))[0]
    return _div_f

def divergence_rev(f):
    def _div_f(params, x):
        jac = jax.jacrev(lambda x: f(params, x))
        return jnp.trace(jac(x))

    return _div_f

In [ ]:
def f(_, x):
    return x**2+x

x = jnp.ones(100)
%timeit divergence_scan_jvp(f)(None, x).block_until_ready() 
%timeit divergence_scan_vjp(f)(None, x).block_until_ready() 
%timeit divergence_fwd(f)(None, x).block_until_ready()  
%timeit divergence_rev(f)(None, x).block_until_ready()  

In [ ]:
%timeit jax.vmap(divergence_scan(network), (None, 0), 0)(params, x).block_until_ready() 
%timeit jax.vmap(divergence_fwd(network), (None, 0), 0)(params, x).block_until_ready() 
%timeit jax.vmap(divergence_rev(network), (None, 0), 0)(params, x).block_until_ready() 

In [ ]:
from jax.scipy.stats import norm
def logp1(x):
    return -0.5 * jnp.sum( x**2 + jnp.log(2 * math.pi), axis=-1)
def logp2(x):
    return jnp.sum(norm.logpdf(x), axis=-1)
key = jax.random.PRNGKey(42)
x = jax.random.normal(key, (4, 2))
jnp.allclose(logp1(x), logp2(x))

In [ ]:
%timeit jax.scipy.linalg.solve(A, b).block_until_ready()  

In [ ]:
%timeit jax.scipy.sparse.linalg.gmres(Ax, b, tol=1e-6)[0].block_until_ready()  

# New test

In [ ]:

np.split(np.array([1,2,3,4]),2,axis=0)